In [1]:
%pip install -q otter-grader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 17.0 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/kateyaknow/homeworke

Cloning into 'homeworke'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 27 (delta 9), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 19.67 KiB | 1.31 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
%cd homeworke

/content/homeworke


In [4]:
# Initialize Otter
import otter
grader = otter.Notebook()

# **Домашнее задание 2.** Метод K-ближайших соседей

<p style="color:red">  
<b>Предупреждение:</b> в данном задании разрешено использовать только NumPy, Pandas и библиотеки для визуализации!
</p>

На семинаре мы познакомились с реализацией KNN в библиотеке scikit-learn, обучили модели для задач классификации и регрессии и оценили их качество на синтетических данных.

Следуя высказыванию одного удивительного человека:

> «What I cannot create, I do not understand.» — Richard Feynman

в этом задании мы реализуем алгоритм k-ближайших соседей на чистом NumPy. Приступим!

In [5]:
from typing import Any, Literal, Self

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:

def convert_pandas_to_numpy(func: callable) -> callable:
    def wrapper(*args: tuple[Any], **kwargs: dict[Any]):
        args = tuple(arg.values if isinstance(arg, (pd.DataFrame, pd.Series)) else arg for arg in args)
        kwargs = {
            key: arg.values if isinstance(arg, (pd.DataFrame, pd.Series)) else arg
            for key, arg in kwargs.items()
        }  # fmt: skip

        return func(*args, **kwargs)

    return wrapper

### **Задание 0.** Предобработка данных

Прочитайте данные из файла `Predict_Calorie_Expenditure_Dataset.csv`, при необходимости удалите пропущенные значения, проведите разведочный анализ данных.  

_Данное задание не оценивается, но его выполнение необходимо для получения полного балла._

_Points:_ 0

In [7]:
data = pd.read_csv('https://raw.githubusercontent.com/ArtemF42/hse-fbb-ml-course-public/refs/heads/main/homeworks/02_nearest_neighbors/Predict_Calorie_Expenditure_Dataset.csv', index_col='id')
data = data.sample(5_000, random_state=42)
data=data.dropna()
print(f"{data.shape = }")
data.head()

data.shape = (5000, 8)


,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
id,,,,,,,,
404846,female,56,163.0,61.0,29.0,108.0,40.8,200.0
580313,male,44,179.0,87.0,13.0,94.0,40.3,64.0
552086,male,41,186.0,84.0,10.0,89.0,39.9,39.0
370876,female,38,161.0,58.0,19.0,98.0,40.6,103.0
239330,male,57,181.0,77.0,9.0,98.0,40.0,52.0


In [8]:
grader.check("Task0")

Task0 results: All test cases passed!

Разобьем данные на обучающую, валидационную и тестовые подвыборки

In [9]:
X, y = data.drop(columns="Calories"), data["Calories"]

X_test = X.sample(frac=0.2, random_state=42)
test_idx = X_test.index
y_test = y.loc[test_idx]

train_idx = X.index.difference(test_idx)
X_train = X.loc[train_idx]
y_train = y.loc[train_idx]

print(f"Размер обучающей выборки:\t{y_train.size} объектов")
print(f"Размер тестовой выборки:\t{y_test.size} объектов")

Размер обучающей выборки:	4000 объектов
Размер тестовой выборки:	1000 объектов


### **Задание 1.** Евклидово расстояние

<p style="color:red">  
<b>Предупреждение:</b> использование циклов <code>for</code> и <code>while</code> в данном задании запрещено!  
</p>  

Пусть матрица $ X_A \in \mathbb{R}^{n \times d} $ содержит объекты с известными целевыми переменными, а матрица $ X_B \in \mathbb{R}^{m \times d} $ — объекты, для которых мы хотим сделать предсказания. Для применения модели KNN нам необходимо научиться считать матрицу расстояний $ D \in \mathbb{R}^{n \times m} $.

В теории мы могли бы, используя вложенные циклы, пройти по строкам первой и второй матриц, считая расстояния между ними, но ярко-красное предупреждение выше не рекомендует нам этого делать. Значит, будем искать другой способ, а для этого посмотрим внимательнее на определение евклидова расстояния:

$$
d(\mathbf{v}, \mathbf{u}) = \sqrt{\sum_{i=1}^{d} (v_i - u_i)^2}, \quad \mathbf{v},\, \mathbf{u} \in \mathbb{R}^d.
$$

Теперь избавимся от корня:

$$
\begin{aligned}
d^2(\mathbf{v}, \mathbf{u})
&= \sum_{i=1}^{d} (v_i - u_i)^2 \\
&= \sum_{i=1}^{d} v_i^2 + \sum_{i=1}^{d} u_i^2 - 2 \sum_{i=1}^{d} v_i u_i \\
&= \lVert \mathbf{v} \rVert^2 + \lVert \mathbf{u} \rVert^2 - 2\,\mathbf{v}\cdot\mathbf{u}.
\end{aligned}
$$

Иными словами, для расчета евклидова расстояния в матричной форме нам всего-то и нужно, что посчитать нормы векторов в исходных матрицах и найти матрицу скалярных произведений. С этим уже можно работать!

**Подсказка:** рассмотрите произведение $ X_A X_B^\top $.

Реализуйте функцию `cdist`, которая вычисляет евклидовы расстояния между строками двух матриц.

_Points:_ 5

In [10]:
@convert_pandas_to_numpy
def cdist(XA: np.ndarray, XB: np.ndarray) -> np.ndarray:
    """Вычисляет матрицу евклидовых расстояний между векторами из двух входных матриц.

    Args:
        XA (np.ndarray): Матрица размера (n, d), содержащая n d-мерных векторов.
        XB (np.ndarray): Матрица размера (m, d), содержащая m d-мерных векторов.

    Returns:
        np.ndarray: Матрица расстояний размера (n, m).
    """
    XA_sq = np.sum(XA**2, axis=1).reshape(-1,1)
    XB_sq = np.sum(XB**2, axis=1).reshape(1,-1)
    return np.sqrt(XA_sq+XB_sq-2*XA@XB.T)

In [11]:
grader.check("Task1")

Task1 results: All test cases passed!

### **Задание 2.** Базовый класс

Наконец, мы готовы приступить к имплементации KNN. Сначала создадим базовый класс, который будет содержать логику обучения и поиска ближайших соседей.

_Points:_ 5

In [12]:
class KNeighborsModel:
    def __init__(self, n_neighbors: int = 5, weights: Literal["uniform", "distance"] = "uniform") -> None:
        """
        Args:
            n_neighbors (int): Количество ближайших соседей K.
            weights (Literal["uniform", "distance"]): Метод агрегации целевых переменных соседей.
                По умолчанию равен uniform.
        """
        self.n_neighbors = n_neighbors
        self.weights = weights

    @convert_pandas_to_numpy
    def fit(self, X: np.ndarray, y: np.ndarray) -> Self:
        """Обучает модель.

        Args:
            X (np.ndarray): Матрица размера (n, d), где n - количество объектов, d - количество признаков.
            y (np.ndarray): Вектор целевых переменных размера (n,).

        Returns:
            Self: Обученная модель.
        """
        assert X.shape[0] == y.shape[0], "`X` и `y` должны иметь одинаковую длину."
        y=y.reshape(-1,)
        self.x_train = X
        self.y_train = y

        return self

    @convert_pandas_to_numpy
    def predict(self, X: np.ndarray) -> np.ndarray:
        neighbors, distances = self._find_neighbors(X)
        return self._aggregate(neighbors, distances)

    def _find_neighbors(self, X: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        """Находит K-ближайших соседей.

        Args:
            X (np.ndarray): Матрица размера (m, d), где m - количество объектов, d - количество признаков.

        Returns:
            tuple[np.ndarray, np.ndarray]: Кортеж (neighbors, distances), где
                neighbors (m, k) - целевые переменные ближайших соседей,
                distances (m, k) - расстояния до ближайших соседей.
        """
        distances = cdist(X,self.x_train)
        idx = np.argsort(distances, axis=1)[:,:self.n_neighbors]
        nearest_distances = np.take_along_axis(distances, idx, axis=1)
        nearest_neighbors = self.y_train[idx]
        return nearest_neighbors, nearest_distances

    def _aggregate(self, neighbors: np.ndarray, distances: np.ndarray) -> np.ndarray:
        if self.weights == "uniform":
            return self._aggregate_uniform(neighbors)
        else:
            weights = 1/(distances+1e-8)
            return self._aggregate_weighted(neighbors, weights)

    def _aggregate_uniform(self, neighbors: np.ndarray) -> np.ndarray:
      #  raise NotImplementedError
        preds = []
        for row in neighbors:
          values, counts = np.unique(row, return_counts=True)
          most_counts = values[np.argmax(counts)]
          preds.append(most_counts)
        return np.array(preds)
    def _aggregate_weighted(self, neighbors: np.ndarray, weights: np.ndarray) -> np.ndarray:
    #    raise NotImplementedError
      classes = np.unique(self.y_train)
      preds = []
      for i in range(neighbors.shape[0]):
        votes ={cls:0.0 for cls in classes}
        for label, w in zip(neighbors[i], weights[i]):
          votes[label]+=np.sum(w[label==neighbors[i]])
        preds.append(max(votes,key=votes.get))
      return np.array(preds)

In [13]:
reg = KNeighborsModel(n_neighbors=1).fit(np.array([[0.3], [0.4], [0.0]]), np.array([1, 1, 0]))
neighbors, distances = reg._find_neighbors(np.array([[0.1]]))
#assert neighbors.item() == 0
#assert np.isclose(distances.item(), 0.1)

In [14]:
reg = KNeighborsModel(n_neighbors=1).fit(np.array([[0.3], [0.4], [0.0]]), np.array([1, 1, 0]))
neighbors, distances = reg._find_neighbors(np.array([[0.1]]))
print(neighbors, distances)

[[0]] [[0.1]]


In [15]:
grader.check("Task2")

Task2 results: All test cases passed!

### **Задание 3**. Метод K-ближайших соседей для задачи регрессии

_Points:_ 2

In [16]:
class KNeighborsRegressor(KNeighborsModel):
    def _aggregate_uniform(self, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors,axis=1)

    def _aggregate_weighted(self, neighbors: np.ndarray, weights: np.ndarray) -> np.ndarray:
        return(np.sum(neighbors*weights,axis=1)/np.sum(weights,axis=1))

In [17]:
neighbors = np.array([[0.4, 0.3, 0.2, 0.1, 0.1]])
weights = 1 / np.array([[1.0, 1.0, 1.0, 3.0, 4.0]])
np.isclose(KNeighborsRegressor()._aggregate_uniform(neighbors), 0.22)

array([ True])

In [18]:
np.isclose(KNeighborsRegressor(weights='distance')._aggregate_weighted(neighbors, weights), 0.267, atol=0.001)

array([ True])

In [19]:
grader.check("Task3")

Task3 results: All test cases passed!

### **Задание 4.** Метрики качества регрессии. Root Mean Squared Error (RMSE)

Для оценки качества модели будем использовать корень из среднеквадратичной ошибки (root mean squared error, RMSE):

$$
\text{RMSE}(\mathbf{y}, \hat{\mathbf{y}}) = \sqrt{\frac{1}{m} \sum_{i=1}^{m} (y_i - \hat{y}_i) ^ 2}.
$$

_Points:_ 2

In [20]:
def root_mean_squared_error(y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
    """Рассчитывает RMSE.

    Args:
        y_true (np.ndarray): Истинные значения целевой переменной.
        y_pred (np.ndarray): Предсказанные значения целевой переменной.

    Returns:
        np.ndarray: Значение метрики RMSE.
    """
    assert y_true.shape == y_pred.shape, "Размеры `y_true` и `y_pred` должны совпадать."
    return np.sqrt(1/y_true.shape[0]*np.sum((y_pred-y_true)**2))

In [21]:
y_true = np.array([0.7, -1.5, 1.4, -1.1])
y_pred = np.array([1.0, 1.7, -1.6, 0.2])
np.isclose(root_mean_squared_error(y_true, y_pred), 2.292, atol=0.001)


np.True_

In [22]:
np.isclose(root_mean_squared_error(y_true, y_true), 0.0)

np.True_

In [23]:
grader.check("Task4")

Task4 results: All test cases passed!

### **Задание 5.** Обучение модели

Теперь обучите модель с параметром `n_neighbors=10` на числовых признаках и оцените ее качество

_Points:_ 2

In [24]:
NUMERIC_FEATURES = X_train.select_dtypes(include=['number']).columns
X_train=X_train[NUMERIC_FEATURES]
X_test=X_test[NUMERIC_FEATURES]
X_test.shape

(1000, 6)

In [25]:
reg = KNeighborsRegressor(n_neighbors=10, weights='distance')
reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)

print(f"RMSE:\t{root_mean_squared_error(y_test, y_pred):.3f}")

RMSE:	7.838


### **Задание 6.** Масштабирование признаков

Вспомните, почему так важно масштабировать признаки при использовании KNN. Затем реализуйте класс `StandardScaler` и примените его к нашим данным.

_Points:_ 2

In [26]:
class StandardScaler:
    @convert_pandas_to_numpy
    def fit(self, X: np.ndarray) -> Self:

        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0,ddof=0)
        self.std_[self.std_==0]=1

        return self

    @convert_pandas_to_numpy
    def transform(self, X: np.ndarray) -> np.ndarray:
        return (X-self.mean_)/self.std_

    @convert_pandas_to_numpy
    def fit_transform(self, X: np.ndarray) -> np.ndarray:
        return self.fit(X).transform(X)

In [27]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [28]:
reg = KNeighborsRegressor(n_neighbors=10, weights = 'distance')
reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)

print(f"RMSE:\t{root_mean_squared_error(y_test, y_pred):.3f}")

RMSE:	6.688


In [29]:
np.allclose(StandardScaler().fit_transform(np.array([[2.7], [2.9], [2.2], [2.0]])), np.array([[0.687], [1.236], [-0.687], [-1.236]]),atol=0.001)

True

In [30]:
grader.check("Task6")

Task6 results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)